## importamos librerias

In [103]:
import scipy
from scipy import stats
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
from sklearn import preprocessing
import copy
import json
import ast
import re
import pandas as pd

# Dataset 'movies_dataset.csv'

In [104]:
df = pd.read_csv('movies_dataset.csv')

C:\Users\agust_5hrd3hq\AppData\Local\Temp\ipykernel_10452\259290390.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv')


In [105]:
#extraemos solo los nombres de las coleccion
patron = r"'name': '([^']*)"
#extreemos nom coleccion
df['coleccion_name'] = df['belongs_to_collection'].str.extract(patron, flags= re.IGNORECASE )
#columna valores individuales
df['coleccion_unico']=df['coleccion_name'].str.split(', ')
#se duplican los registros
df = df.explode('coleccion_unico').reset_index(drop= True)
#se elimina la columna 'coleccion_name
df.drop('coleccion_name', axis=1, inplace= True) 



In [107]:
# Eliminar los valores NaN de la columna 'production_companies'
df = df.dropna(subset=['production_companies'])
# convertimos en lista los valores de la columna 'production_companie'
df['production_companies'] = df['production_companies'].apply(ast.literal_eval)
# Filtrar los valores booleanos en la columna 'production_companies'
df = df[df['production_companies'].apply(lambda x: isinstance(x, list))]
#extreaemos solamente los nombres de las companies con la funcion extract_comapanies_name
def extract_companies_name(companie_list):
    comp_name = [cpname['name'] for cpname in companie_list]
    return ', '.join(comp_name)
# aplicamos la función a la columna
df['companie_name'] = df['production_companies'].apply(extract_companies_name)
# separamos los valores unicos de la columna 'companpanie_name'
df['companie_unico'] = df['companie_name'].str.split(', ')

#explode
df = df.explode('companie_unico').reset_index(drop = True)

#vamos a elimina la columna companie_name
df.drop('companie_name', axis=1, inplace=True)
#mostrar el dataset
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,coleccion_unico,companie_unico
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Pixar Animation Studios
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,TriStar Pictures
2,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Teitler Film
3,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Interscope Communications
4,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection,Warner Bros.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82932,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaN,
82933,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,NaN,Sine Olivia
82934,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,NaN,American World Pictures
82935,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,NaN,Yermoliev


In [108]:

# convertimos en lista los valores de la columna 
df['production_countries'] = df['production_countries'].apply(ast.literal_eval)

#extreaemos solamente los nombres de las companies con la funcion extract_countries_name
def extract_countries_name(ctry_list):
    contry_name = [ctname['name'] for ctname in ctry_list]
    return ', '.join(contry_name)
# aplicamos la función a la columna
df['country_name'] = df['production_countries'].apply(extract_countries_name)
# separamos los valores unicos de la columna 
df['unique_country_name'] = df['country_name'].str.split(', ')

#explode
df = df.explode('unique_country_name').reset_index(drop = True)

#vamos a elimina la columna
df.drop('country_name', axis=1, inplace=True)
#mostrar el dataset

In [109]:
#convertimos en lista los valores de la columna 'genres'
df['genres']=df['genres'].apply(ast.literal_eval)

# aquí vamos a extaer solamente los nombres de los géneros con la funcin 'df extract_genres_name()'
def extract_genres_name(genres_list):
    genre_name = [genre['name'] for genre in genres_list]
    return ', '.join(genre_name)
#ahora vamos a plicar la función a la columna
df['genres'] = df['genres'].apply(extract_genres_name)

# separamos los valores unicos de la columna 'genres'
df['genre_unico'] = df['genres'].str.split(', ')

#explode
df = df.explode('genre_unico').reset_index(drop = True)

#vamos a elimina la columna 'genres'
df.drop('genres', axis=1, inplace=True)

In [110]:
# IDIOMA
# convertimos en lista los valores de la columna 'spoken_languages'
df['spoken_languages'] = df['spoken_languages'].apply(ast.literal_eval)

#extreaemos solamente los nombres de las companies con la funcion extract_comapanies_name
def extract_spl_name(idioma_list):
    idioma_name = [nameleng['name'] for nameleng in idioma_list]
    return ', '.join(idioma_name)
# aplicamos la función a la columna
df['idioma_name'] = df['spoken_languages'].apply(extract_spl_name)
# separamos los valores unicos de la columna 
df['unique_idioma_name'] = df['idioma_name'].str.split(', ')

#explode
df = df.explode('unique_idioma_name').reset_index(drop = True)

#vamos a elimina la columna 
df.drop('idioma_name', axis=1, inplace=True)
#mostrar el dataset

In [112]:
# eliminamos los nulos de las columnas 'revenue' y 'budget'
df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace= True)

#verificamos si realmente no hay nulos. Nos debe dar de valos 0 (cero)
print('Revenue nulos: ', df.revenue.isna().sum())
print('budget nulos: ',df.budget.isna().sum())

Revenue nulos:  0
budget nulos:  0


In [113]:
#Los valores nulos del campo release_date deben eliminarse.
# De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
df['release_date'] =pd.to_datetime(df['release_date'], errors='coerce')
df['release_date']= pd.to_datetime(df['release_date']).dt.strftime('%Y-%m-%d')

#Ahora si podemos eliminar los valores nulos
df=df.dropna (subset= ['release_date'])
df['release_date'] = pd.to_datetime(df['release_date'])

#Creamos la columna release_year donde extraerán el año de la fecha de estreno.
df['release_year'] = df['release_date'].dt.year
#mostramos como nos queda las dos columnas
df[['release_date', 'release_year']].head(10)

C:\Users\agust_5hrd3hq\AppData\Local\Temp\ipykernel_10452\748416107.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'] = pd.to_datetime(df['release_date'])
C:\Users\agust_5hrd3hq\AppData\Local\Temp\ipykernel_10452\748416107.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = df['release_date'].dt.year


,release_date,release_year
0,1995-10-30,1995
1,1995-10-30,1995
2,1995-10-30,1995
3,1995-12-15,1995
4,1995-12-15,1995
5,1995-12-15,1995
6,1995-12-15,1995
7,1995-12-15,1995
8,1995-12-15,1995
9,1995-12-15,1995


In [114]:
df = df[df['release_date'] != 'sin dato']
df['release_date'] = pd.to_datetime(df['release_date'])

In [115]:
# Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, 
# dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

df['revenue'] = pd.to_numeric(df['revenue'], errors = 'coerce')
df['budget'] = pd.to_numeric(df['budget'], errors ='coerce')

# se hace el calculo para la culumna return que es el retorno de inversion
df['return']=df['revenue']/df['budget']
df['return'].fillna(0, inplace= True)
df['return']

0         12.451801
1         12.451801
2         12.451801
3          4.043035
4          4.043035
            ...    
422129     0.000000
422130     0.000000
422131     0.000000
422132     0.000000
422133     0.000000
Name: return, Length: 421964, dtype: float64

In [116]:
# Eliminamos las columnas que no vamos a utilizar
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df = df.drop(columns_to_drop, axis=1)


In [126]:
#guardamos todo en un archivo
df.to_csv('movies_transformado.csv', index=False)

# Dataset 'credits.csv'

In [133]:
creditos = pd.read_csv('credits.csv')

In [129]:
# Necestimos extraer el nombre de los directores que se encuentran en la columna crew
creditos

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [134]:
# convertimos en lista los valores de la columna
creditos['crew'] = creditos['crew'].apply(ast.literal_eval)

#extreaemos solamente los nombres 
def extract_crew_name(crew_list):
    c_name = [namecrew['name'] for namecrew in crew_list]
    return ', '.join(c_name)
# aplicamos la función a la columna
creditos['crew_name'] = creditos['crew'].apply(extract_crew_name)
# separamos los valores unicos de la columna
creditos['unique_crew_name'] = creditos['crew_name'].str.split(', ')

#explode
creditos = creditos.explode('unique_crew_name').reset_index(drop = True)

#vamos a elimina la columna 'crew_name'
creditos.drop('crew_name', axis=1, inplace=True)
#mostrar el dataset

In [136]:
creditos.to_csv('creditos_transformado.csv', index=False)

# Concatenamos el archivo 'movies_transformado.csv' con el archivo 'creditos_gransformado.csv'

In [138]:
# Cargar los archivos CSV
movies_df = pd.read_csv('movies_transformado.csv')
creditos_df = pd.read_csv('creditos_transformado.csv')

# Concatenar los dataframes
concatenated_df = pd.concat([movies_df, creditos_df], axis=1)

#guardamos

concatenated_df.to_csv('concatenado.csv', index= False)